In [1]:
import yfinance as yf
import pandas as pd

Data Preparation

In [2]:
nifty50 = yf.Ticker('^NSEI')
nifty50 = nifty50.history(start='2014-01-01', end='2024-10-15')
#the volume here is (The actual volume / 1000)

In [3]:
del nifty50["Dividends"]
del nifty50["Stock Splits"]

In [4]:
nifty50["Tommorow"] = nifty50['Close'].shift(-1)

In [5]:
nifty50['Target'] = (nifty50['Tommorow'] > nifty50['Close']).astype(int)

In [6]:
nifty50 = nifty50.drop(nifty50.index[-1])

Making The Price Predictor

In [7]:
from sklearn.ensemble import RandomForestRegressor
nifty50["Price_Change"] = nifty50["Tommorow"] - nifty50["Close"]
train = nifty50.iloc[:-100]
test = nifty50.iloc[-100:]
Predictors = ['Close', 'Volume', 'Open', 'High', 'Low']

regressor = RandomForestRegressor(n_estimators=100, min_samples_split=100, random_state=1)
regressor.fit(train[Predictors], train["Price_Change"])

price_change_preds = pd.Series(regressor.predict(test[Predictors]))


from sklearn.metrics import root_mean_squared_error
mse = root_mean_squared_error(test["Price_Change"], price_change_preds)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 261.6617914089675


Making The Classifier

In [8]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1)
train = nifty50.iloc[:-100]
test = nifty50.iloc[-100:]
Predictors = ['Close', 'Volume', 'Open', 'High', 'Low']
model.fit(train[Predictors], train["Target"])

preds = pd.Series(model.predict(test[Predictors]))

from sklearn.metrics import precision_score
precision_score(test['Target'], preds)

1.0

Saving The Models Into Pickle Files

In [9]:
import joblib
joblib.dump(model, 'stockpred.pkl')
joblib.dump(regressor, 'price_change_predictor.pkl')


['price_change_predictor.pkl']